In [ ]:
!pip install kora -q
!pip3 install segments-ai -q
!pip install efficientnet_pytorch -q
from kora import drive
drive.link_nbs()
import efficientnet_pytorch
import neuralnets
import dataset_loader
from PIL import Image
import torch 
from torchvision import models
from torchvision.datasets import CocoDetection
import torchvision.transforms as transforms
import segments
import numpy as np
import os

# Try to use gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

First we define a utility class which can load a model withs respect to:

*   Used archictecture ``` arch ```
*   Dataset used for training ``` dataset ```
*   Optimizer used during the training ``` optimizer ```

The model should all be placed in the ``` model_root ``` folder.
For each supported model (``` efficientnet ```, ``` resnet ```) a corresponding folder with the same name needs to be placed in the root folder which results in the following structure: 
```
path/to/model_root -- efficientnet
                   |
                   -- resnet
```
The model should be be placed in these folders and be named with this sheme:

``` dataset + "_" + optimizer + ".h5" ```

for example:

``` "minc_adam.h5" ```


In [ ]:
class MaterialClassifier:
  @staticmethod
  def get_classifier(model_root, arch, dataset, optimizer):
    
    if dataset == 'minc':
      num_classes = 23
    else:
      num_classes = 4
    if arch == 'efficientnet':
      model = efficientnet_pytorch.EfficientNet.from_name("efficientnet-b1", num_classes=num_classes)
    elif arch == 'resnet':
      model = models.wide_resnet50_2(pretrained=False, num_classes=num_classes)
    model_dir = os.path.join(model_root, arch)
    model_name = dataset + "_" + optimizer + ".h5"
    model_path = os.path.join(model_dir, model_name)
    state_dict = torch.load(model_path, map_location='cuda:0')#,map_location=torch.device())
    ret = model.load_state_dict(state_dict, strict=False)
    model.eval()
    return model.to(device)

We assume that the segmented data is given in coco format. This way we can utilize the already existing coco loader. (For more details: ``` dataset_loader.ipynb```)

The ```root``` and ```annFile``` need to be set.

Mappings from the MINC-2500 and the OpenSurfaces subset datasets to the ground truth category ids are given by ```minc_to_common``` and ```os_to_common```.

In [ ]:
root = "/content/drive/MyDrive/Datasets/segments/computervision2021_Indoor/v2.0.2"
annFile = "/content/drive/MyDrive/Datasets/Indoor-v2.0.2_coco.json"
test_dataset = dataset_loader.CocoLoader(root, annFile)
data_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=1, 
                                          shuffle=False)
minc_to_common = {0: 4, #"Granite_marble", 
        1: 7, #"Concrete", 
        2: 5, #"Tile", 
        3: 3, #"Wood"
        }

os_to_common = {2: 4, #"Granite_marble", 
        1: 7, #"Concrete", 
        3: 5, #"Tile", 
        0: 3, #"Wood"
        }
        
minc_selector = [15,18,19,22]
os_selector = [0,1,2,3]


Method to evaluate a model on a dataset. Following parameters needs to be set:



*   ``` model ``` - Use ``` MaterialClassifier.get_classifier(...) ``` to get a model
*   ```data_loader``` - Use ``` data_loader ``` from the last block
*   ```crop_size``` - Patch size (``` crop_size x crop_size ```) used for classification
*   ```dataset_to_common``` - Mapping from the dataset category ids to the classifier ids (i.e. use ```minc_to_common``` or ```os_to_common```)
*   ```num_classes``` - Number of different classes which get estimated by the ``` model ```
* ```select_classes``` - Classes of intereset (i.e. use ```minc_selector``` or ``` os_selector ```)

The method will return two types of accuracy:
* Patch accuracy - Ratio of correct patches vs. all patches
* Segment accuracy - Ratio of correct segments vs. all segments. A segment materials is estimated by a majority vote from all patches inside this segment.





In [ ]:
def classify_segments(model, data_loader, crop_size, dataset_to_common, num_classes, select_classes):
  trans = transforms.Compose([
          transforms.ToTensor(),
          transforms.Normalize([0.44074593, 0.44086692, 0.44087528], [0.2285, 0.2284, 0.2285])
        ])
  common_to_dataset = dict([reversed(i) for i in dataset_to_common.items()])

  total_segments = 0
  correct_segments = 0
  total_patches = 0
  correct_patches = 0
  with torch.no_grad():
    for image, segmentation_mask, name in data_loader:
      np_image = np.asarray(image[0])
      image = torch.transpose(trans(np_image),0,2)
      segmentation_mask = segmentation_mask[0]
      height, width = segmentation_mask.size()
      if width < crop_size or height < crop_size:
        raise ValueError("Width of height of image is smaller then crop size")

      sample_count_width = int(width/crop_size)-1
      sample_count_height = int(height/crop_size)-1

      crop_area = crop_size*crop_size
      crop_segment_labels = []
      crops = {int(category_id):[] for category_id in torch.unique(segmentation_mask)}

      for i in range(sample_count_height):
        for j in range(sample_count_width):
          crop_segment_label = segmentation_mask[i*crop_size + int(crop_size/2),
                                                 j*crop_size + int(crop_size/2)]
          if crop_segment_label in [2,3,4,5,6,7]:
            left = i * crop_size
            right = (i + 1) * crop_size
            upper = j * crop_size
            lower = (j + 1) * crop_size
            crop = image[left:right, upper:lower,:]
            crops[int(crop_segment_label)].append(crop)
      for category in crops.keys():
        patch_count = len(crops[category])
        if patch_count > 0:
          input_crops = torch.stack(crops[category])
          input_crops = torch.transpose(input_crops, 1,3)

          images = input_crops.to(device)
          outputs = model(images)
          _, predicted = outputs.max(1)
          category_count = torch.bincount(predicted, minlength=num_classes)[select_classes]

          reduced_class = torch.argmax(category_count)
          class_of_segment = dataset_to_common[reduced_class.item()]

          if class_of_segment == category:
            correct_segments += 1
          total_segments += 1

          total_patches += patch_count
          if category in common_to_dataset.keys():
            correct_patches += category_count[common_to_dataset[category]].item()

  accuracy_segments = correct_segments / total_segments
  accuracy_patches = correct_patches / total_patches
  return accuracy_segments, accuracy_patches

The following code will process the tests over 16 combinations of different architectures (```archs```), datasets (```datasets```), optimizers (```optimizers```).

The script expact that all models are places in subdirs of the ```model_dir``` folder.
You will need one subdir per architecture in ```archs```, named as the architecture. (i.e. ``` /path/to/model_dir/efficientnet ``` or ```/path/to/model_dir/resnet ```).

For each combination the script will print the results like this:
```
Architectur: efficientnet, Dataset: os224flipped, Optimizer: adam -- Segment Accuracy: 0.523077, Patch Accuracy: 0.528325
Architectur: efficientnet, Dataset: os224flipped, Optimizer: sgd -- Segment Accuracy: 0.446154, Patch Accuracy: 0.495746
```

In [ ]:
archs = ["efficientnet" , "resnet"]
datasets = ["minc", "os128", "os224", "os224flipped"]
optimizers = ["adam", "sgd"]
model_root = "/content/drive/MyDrive/models"

for arch in archs:
  for dataset in datasets:
    for optimizer in optimizers:
      if '224' in dataset:
        crop_size = 224
      else:
        crop_size = 128
      if dataset == 'minc':
        num_classes = 23
        select_classes = minc_selector
        dataset_to_common = minc_to_common
      else:
        num_classes = 4
        select_classes = os_selector
        dataset_to_common = os_to_common
      model = MaterialClassifier.get_classifier(model_root, arch, dataset, optimizer)
      seg_acc, patch_acc = classify_segments(model, 
                                             data_loader, 
                                             crop_size, 
                                             dataset_to_common, 
                                             num_classes, 
                                             select_classes)
      print("Architectur: %s, Dataset: %s, Optimizer: %s -- Segment Accuracy: %f, Patch Accuracy: %f" % (arch, dataset, optimizer, seg_acc, patch_acc))

